# Using Callbacks in Keras

In this notebook, we well see how to use pre-defined and custom callbacks in Keras for tasks such as chekpointing, learning rate scheduling, etc.

We'll use the same simple dataset and linear model of the previous notebook.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

##### Download the Auto-MPG dataset
 
Let's use the Auto-MPG dataset (seen in a previous notebook).

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

dataset = pd.read_csv(url, names=column_names, na_values='?', comment='\t', sep=' ', skipinitialspace=True)
dataset = dataset.dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

##### Build the model

Let's build a simple linear regression model (seen in a previous notebook) to test different callbacks during its training.

We use a `get_model()` function so that we can re-create the model from scratch multiple times with a single instruction.

In [ ]:
def get_model():
    normalizer = preprocessing.Normalization(input_shape=[9,])
    normalizer.adapt(np.array(train_features))
    
    model = keras.Sequential([
        normalizer,
        layers.Dense(units=1)
    ])
    
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=0.1),
        loss='mse', metrics=['mae', 'mse']
    )
    
    return model

## Early Stopping callback

Let's use the early stopping callback to stop training when it reaches stability.

The `monitor` parameter specifies the loss/metric to be monitored, and the `patience` parameters specifies the number of non-improving epochs to wait before stopping. 

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# re-create the model to restart training every time
model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2, callbacks=[es_callback])

As you can see, the training stopped after about 60/70 epochs, rather than running for the entire 200 epochs specified in `fit()`.

## Checkpoint Callback

Let's add a second callback to save a model checkpoint after every epoch. Notice that we can pass multiple callbacks at the same time to `fit()`.

In [ ]:
cp_callback = keras.callbacks.ModelCheckpoint(
    './callback_test_chkp/chkp_{epoch:02d}',
    # './callback_test_chkp/chkp_best',
    monitor='val_loss',
    verbose=0, 
    save_best_only=False,
    # save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)

In [ ]:
model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2,
                                callbacks=[es_callback, cp_callback])

##### Restore a saved checkpoint

Let's try loading back two different models, and let's evaluate them on training data.

In [ ]:
model_epoch1 = keras.models.load_model('./callback_test_chkp/chkp_01')
model_epoch1.evaluate(train_features, train_labels,)

In [ ]:
model_epoch10 = keras.models.load_model('./callback_test_chkp/chkp_10')
model_epoch10.evaluate(train_features, train_labels,)

## Learning Rate Scheduling

Let's try to change the learning rate by reducing it by 0.01 after every epoch. This is just to demonstrate LR scheduling, it is not a particularly useful scheduling mechanism.

In [ ]:
def my_schedule(epoch, lr):
    return max(lr - 0.01, 0.01)

Test if the schedule works for different input LR values.

In [ ]:
print(my_schedule(1, 0.05))
print(my_schedule(1, 0.01))

In [ ]:
lr_callback = keras.callbacks.LearningRateScheduler(my_schedule, verbose=1)

In [ ]:
model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2,
                                callbacks=[lr_callback, es_callback])

## Custom Callback N.1

Let's write a simple custom callback that logs the loss and metrics values after every batch, epoch, etc.

In [ ]:
class CustomLogger(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        print("Starting training; log content: {}".format(logs))

    def on_train_end(self, logs=None):
        print("Stop training; got log content: {}".format(logs))

    def on_epoch_end(self, epoch, logs=None):
        print("End epoch {} of training; log content: {}".format(epoch, logs))

    def on_train_batch_end(self, batch, logs=None):
        print("...Training: end of batch {}; log content: {}".format(batch, logs))


In [ ]:
log_callback = CustomLogger()

model = get_model()
history = model.fit(train_features, train_labels, epochs=200,
                                verbose=0, #verbose=0 to avoid mixing our prints and those of standard keras
                                validation_split = 0.2,
                                callbacks=[log_callback]
)

# Custom Callback N. 2

Let us write another example of custom callback. This time, let's implement a custom early stopping mechanism on a pre-defined Validation MAE value.

In [ ]:
class MyEarlyStopping(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if(logs['val_mae']< 10.0):
            print("\nReached MAE < 10.0, so cancelling training!")
            self.model.stop_training = True


In [ ]:
my_es_callback = MyEarlyStopping()

model = get_model()
history = model.fit(train_features, train_labels, epochs=200, validation_split = 0.2, callbacks=[my_es_callback])